In [3]:
import pandas as pd# pass in column names for each CSV and read them using pandas. 
# Column names available in the readme file
#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('Dane/ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')
#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('Dane/ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')
#Reading items file:
i_cols = ['movie_id', 'movie_title' ,'release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('Dane/ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_train = pd.read_csv('Dane/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('Dane/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [2]:
users.head()

NameError: name 'users' is not defined

In [3]:
items.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [77]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
# macierze traktowane jako parametry funkcji celu
import numpy as np
import random

In [18]:
# uczenie przez gradient spadku - daje macierz prognoz dla elementów pustych
k = 5
U = np.random.uniform(-0.001,0.001, (len(users),k))
I = np.random.uniform(-0.001,0.001, (k, len(items)))
learning_rate = 0.01
for e in range(10):
    for i in range(len(ratings[:100000])):
        #print(ratings['user_id'][i])
        user_id = ratings['user_id'][i]-1
        movie_id = ratings['movie_id'][i]-1
        us = U[user_id]
        it = I[:,movie_id]
        dCdu = 2*(ratings['rating'][i] - np.dot(us,it))*(-it) #pochodne cząstkowe
        dCdi = 2*(ratings['rating'][i] - np.dot(us,it))*(-us)
        U[user_id,:] = U[user_id,:] - learning_rate * dCdu
        I[:,movie_id] = I[:,movie_id] - learning_rate * dCdi #aktualizacja
        #print(max(U[user_id]))
        #print(max(I[:,movie_id]))
        
    cost = 0 #odchylenie predykcji
    for i in range(len(ratings[:])):
        user_id = ratings['user_id'][i]-1
        movie_id = ratings['movie_id'][i]-1
        cost = cost + (ratings['rating'][i] - np.dot(U[user_id], I[:,movie_id]))**2   
    print(cost/len(ratings[:100000]))
    #print(max(U[user_id]), max(I[:,movie_id]))

2.50346039938
1.00704140104
0.938134906521
0.9187785222
0.909620377423
0.90128555023
0.883885403569
0.857486890639
0.837843489643
0.820497574796


In [28]:
# ocena skoru modelu
k = 80
learning_rate = 0.01
U = np.random.uniform(-0.001,0.001, (len(users),k))
I = np.random.uniform(-0.001,0.001, (k, len(items)))

for e in range(10):
    for i in range(len(ratings_train)):
        #print(ratings_train['user_id'][i])
        user_id = ratings_train['user_id'][i]-1
        movie_id = ratings_train['movie_id'][i]-1
        us = U[user_id]
        it = I[:,movie_id]
        dCdu = 2*(ratings_train['rating'][i] - np.dot(us,it))*(-it) #pochodne cząstkowe
        dCdi = 2*(ratings_train['rating'][i] - np.dot(us,it))*(-us)
        U[user_id,:] = U[user_id,:] - learning_rate * dCdu
        I[:,movie_id] = I[:,movie_id] - learning_rate * dCdi #aktualizacja
        
    cost = 0 #odchylenie predykcji
    for i in range(len(ratings_train[:])):
        user_id = ratings_train['user_id'][i]-1
        movie_id = ratings_train['movie_id'][i]-1
        cost = cost + (ratings_train['rating'][i] - np.dot(U[user_id], I[:,movie_id]))**2   
    #print('Epoch', e, 'Training cost:', cost/len(ratings_train))
    
    # ocena skoru modelu
    cost2 = 0 #odchylenie predykcji
    for i in range(len(ratings_test[:])):
        user_id = ratings_test['user_id'][i]-1
        movie_id = ratings_test['movie_id'][i]-1
        cost2 = cost2 + (ratings_test['rating'][i] - np.dot(U[user_id], I[:,movie_id]))**2 #U,I -przekaz inf do zb test
    print('Epoch', e+1, 'Training cost:', cost/len(ratings_train), ' Testing cost:', cost2/len(ratings_test))


Epoch 0 Training cost: 8.77942076564  Testing cost: 6.91984057494
Epoch 1 Training cost: 1.4956675133  Testing cost: 1.65322669939
Epoch 2 Training cost: 1.09538504795  Testing cost: 1.14299250595
Epoch 3 Training cost: 1.00478831348  Testing cost: 1.06453407407
Epoch 4 Training cost: 0.954585032093  Testing cost: 1.03412224241
Epoch 5 Training cost: 0.918155479886  Testing cost: 1.01830684184
Epoch 6 Training cost: 0.88320486589  Testing cost: 1.00511218829
Epoch 7 Training cost: 0.844109490004  Testing cost: 0.990293516066
Epoch 8 Training cost: 0.806382117202  Testing cost: 0.977302861174
Epoch 9 Training cost: 0.771059878643  Testing cost: 0.968520878803


In [29]:
# Factorization machines
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
ratings_user = pd.get_dummies(ratings['user_id'])
ratings_movie = pd.get_dummies(ratings['movie_id'])
#ratings_rating = pd.get_dummies(ratings['rating'])
r_concated = pd.concat([ratings_user, ratings_movie], axis=1)
print( ratings_user.shape, ratings_movie.shape, r_concated.shape)

(100000, 943) (100000, 1682) (100000, 2625)


In [51]:
#ratings_other_movies = np.zeros((len(ratings), len(items)))

In [6]:
from pandas import pivot_table
ratings_other_movies = pivot_table(ratings,
                                  values= 'rating',
                                  index = ['user_id'],
                                  columns = ['movie_id'],
                                  aggfunc = np.count_nonzero )  
ratings_other_movies = ratings_other_movies.iloc[ratings['user_id']-1].fillna(0)
ratings_other_movies

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
suma = ratings_other_movies.sum(axis=0)
ratings_other_movies2 = ratings_other_movies / suma
ratings_other_movies2.shape

(100000, 1682)

In [9]:
r_concated2 = np.concatenate([np.array(r_concated), ratings_other_movies2], axis=1)
#print(r_concated.shape, b.shape)

In [10]:
r_concated2.shape

(100000, 4307)

In [11]:
r_previous = pd.concat([ratings.sort_values(["user_id","unix_timestamp"]),
                  ratings.sort_values(["user_id","unix_timestamp"])
                  .groupby("user_id")["movie_id"].shift(1).rename("previous_movie")],
                 axis=1)
r_previous.shape

(100000, 5)

In [12]:
r_stamps = (ratings.unix_timestamp-ratings.unix_timestamp.min())/(ratings.unix_timestamp.max()-ratings.unix_timestamp.min())
#r_stamps[:10]

In [13]:
#r_previous.previous_movie.shape
r_previous2 = pd.get_dummies(r_previous.previous_movie).fillna(0)
r_previous2.shape

(100000, 1679)

In [146]:
type(r_stamps)

pandas.core.series.Series

In [14]:
r_concated3 = np.concatenate([r_concated2, np.array(r_previous2), np.array(r_stamps).reshape(-1,1)], axis=1)
print(r_concated2.shape, r_previous2.shape, r_stamps.shape)

(100000, 4307) (100000, 1679) (100000,)


In [150]:
len(r_concated3)#.shape

100000

In [ ]:
#w0, w wektor, v macierz losowe liczby inicjalizacja

In [15]:
k = 5
w0 = 0 #np.random.uniform(-0.000005,0.000005)
w = np.random.uniform(-0.000005,0.000005, (r_concated3.shape[1]))
V = np.random.uniform(-0.000005,0.000005,(r_concated3.shape[1],k))
#V -= (-1) * learning_rate * 2 * err * (np.dot(x,V)*x.reshape(-1,1) - V*(x.reshape(-1,1)**2))

In [ ]:
learning_rate = 0.001
for e in range(100):
    for i in range(r_concated3[:80000].shape[1]):
        x = r_concated3[i]
        y = ratings.rating[i]
        y_pred = w0 + np.dot(w,x) + 0.5 * (np.dot(x,V)**2 - np.dot(x**2,V**2)).sum()
        err = (y - y_pred)
        w0 = w0 - (-1) * learning_rate * 2 * err * 1
        w = w - (-1) * learning_rate * 2 * err * x
        V -= (-1) * learning_rate * 2 * err * (np.dot(x,V)*x.reshape(-1,1) - V*(x.reshape(-1,1)**2))
 

    cost = 0 #odchylenie predykcji
    for i in range(80000):
        x = r_concated3[i]
        y = ratings.rating[i]
        y_pred = w0 + np.dot(w,x) + 0.5 * (np.dot(x,V)**2 - np.dot(x**2,V**2)).sum()
        err = (y - y_pred)
        cost = cost + err**2   
    #print('Epoch', e, 'Training cost:', cost/r_concated3.shape[1])
    
    cost2 = 0 
    for i in range(20000):
        x = r_concated3[80000:][i]
        y = ratings.rating[80000:].iloc[i]
        y_pred = w0 + np.dot(w,x) + 0.5 * (np.dot(x,V)**2 - np.dot(x**2,V**2)).sum()
        err = (y - y_pred)
        cost2 = cost2 + err**2   
    print('Epoch', e, 'Training cost:', cost/80000, "test: ", cost2/20000)

Epoch 0 Training cost: 1.17887390376 test:  1.19839929896
